In [ ]:
# Imports
import os

import warnings
warnings.filterwarnings('ignore')

import pandas as pd
from bs4 import BeautifulSoup as bs

In [ ]:
# --------------------------------------------------------------------------------------
# Vereist list met folder van landen bv: ['ParlaMint-NL.TEI', "ParlaMint-BE.TEI"]
#
# ---Return list met dict [{Land1}, {Land 1}]---
#
# Voorbeeld voor land MET jaarfolders:
# item 1 --> 'land folder': [land xml] bv: 'ParlaMint-NL.TEI': ['ParlaMint-NL.xml']
# item rest --> 'ParlaMint-NL.TEI\\2014': ['ParlaMint-NL_2014-04-16-tweedekamer-2.xml', '...']
#
# Voorbeeld met land ZONDER jaarfolder:
# items (alle) --> {'ParlaMint-BG.TEI': ['ParlaMint-BG.xml', 'ParlaMint-BG_2014-10-27.xml', '...'}
#
# Ofwel len(dict) > 1 bij landen met jaarfolders, anders zit speciale file inbegrepen als eerste in dict
#
#  Path vinden met os.path.join(key, value[x])
# --------------------------------------------------------------------------------------

def get_xml_files(country_selection):
    os.chdir(r"C:\Users\Asher\Documents\School\_Scriptie\Data\Data_Extract")
    
    country_return_list = []

    # loop door alle folders die hierboven zijn geprint
    for country in os.listdir():

        # filter op specifiek land (IN BOX 2)
        if country in country_selection:
            paths_dict = {}

            # ga door alle inhoud van de landfolder heen
            for root, dirs, files in os.walk(country):
                file_data = []
                
                # loop door files van een folder
                for file in files:

                    #filter alleen de xml files
                    if ".xml" in file and not "~" in file:
                        file_data.append(file)

                # filter onzin uit de dict voor makkelijkere processing later
                if not "Schema" in root:
                    paths_dict[root] = file_data

            country_return_list.append(paths_dict)
        
    return country_return_list

In [ ]:
# functie testruimte
selected_countries = ["ParlaMint-NL.TEI"]

answer = get_xml_files(selected_countries)

print(answer)

In [ ]:
def extract_wordcount(root, file):
    os.chdir(r"C:\Users\Asher\Documents\School\_Scriptie\Data\Data_Extract")
    
    path = os.path.join(root, file)
    
    # lees het bestand
    with open(path, "r", encoding="utf-8") as file:
        # read each line in the file, readlines() returns a list of lines
        content = file.readlines()
        # combine the lines in the list into a string
        content = "".join(content)
        bs_content = bs(content, "lxml")
        
    words = bs_content.find_all("measure")[-1].get("quantity")
    
    return words

In [ ]:
# functie testruimte
root, file = "ParlaMint-NL.TEI\\2014", "ParlaMint-NL_2014-04-16-tweedekamer-2.xml"

test_json = extract_wordcount(root, file)

print(test_json)

In [ ]:
def get_wordcount_df(selected_countries):
    
    # zet de df op voor de wordcount
    wordcount_df = pd.DataFrame({'country': [],
                                 'filename': [],
                                 'words': []})
    
    # vraag alle xml files op van de gegeven land(en)
    countries = get_xml_files(selected_countries)
    
    # loop door elk land heen
    for country in countries:
        
        # split hoofdfolder van mogelijke subfolders van jaren
        rootfolder = (k := next(iter(country)), country.pop(k))
        
        # country label (bv: GB of NL of BE)
        cty = rootfolder[0].replace("ParlaMint-", "").replace(".TEI", "")
        
        # check of er wel jaarfolders zijn
        if len(rootfolder[1]) == 1:

            for year_folder in country:

                for file in country[year_folder]:
                    
                    wordcount_df.loc[len(wordcount_df.index)] = [cty, 
                                                                 file, 
                                                                 extract_wordcount(year_folder, file)]

        # geen jaarfolders
        else:

            # haal de personendata uit de filelijst
            rootfolder[1].pop(0)

            for file in rootfolder[1]:                
                
                wordcount_df.loc[len(wordcount_df.index)] = [cty, 
                                                             file, 
                                                             extract_wordcount(rootfolder[0], file)]

        print("---------------------------------------------------------------\n\n")
        print(cty + " is uploaded to elastic\n\n")
        print("---------------------------------------------------------------\n\n")
    
    return wordcount_df

In [ ]:
selected_countries = ['ParlaMint-BE.TEI',
                      'ParlaMint-BG.TEI',
                      'ParlaMint-CZ.TEI',
                      'ParlaMint-DK.TEI',
                      'ParlaMint-ES.TEI',
                      'ParlaMint-FR.TEI',
                      'ParlaMint-GB.TEI',
                      'ParlaMint-HR.TEI',
                      'ParlaMint-HU.TEI',
                      'ParlaMint-IS.TEI',
                      'ParlaMint-IT.TEI',
                      'ParlaMint-LT.TEI',
                      'ParlaMint-NL.TEI',
                      'ParlaMint-LV.TEI',
                      'ParlaMint-PL.TEI',
                      'ParlaMint-SI.TEI',
                      'ParlaMint-TR.TEI']

# selected_countries = ["ParlaMint-BG.TEI"]

# selected_countries = ["ParlaMint-NL.TEI"]

test_wordcount_df = get_wordcount_df(selected_countries)

In [ ]:
display(test_wordcount_df)

In [ ]:
csv_loc = 'C:/Users/Asher/Documents/School/_Scriptie/Data/xml_word_counts.csv'

# test_wordcount_df.to_csv(csv_loc, index=False)

In [ ]:
df_words_import = pd.read_csv(csv_loc)

display(df_words_import)